In [1]:
pip show sqlalchemy

Name: SQLAlchemy
Version: 2.0.25
Summary: Database Abstraction Library
Home-page: https://www.sqlalchemy.org
Author: Mike Bayer
Author-email: mike_mp@zzzcomputing.com
License: MIT
Location: c:\users\nagesh\anaconda3\envs\cubes\lib\site-packages
Requires: greenlet, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install sqlalchemy

In [3]:
!pip install cubes[all]

  Using cached Jinja-1.2.tar.gz (252 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [7 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\Nagesh\AppData\Local\Temp\pip-install-26w2om3f\jinja_9bd8e5acff1541c48b80d62d19bd5818\setup.py", line 28
          except DistutilsError, e:
                               ^
      SyntaxError: invalid syntax
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv

In [5]:
engine = create_engine('sqlite:///data.sqlite')

In [10]:
import sqlalchemy

def create_table_from_csv(connectable, file_name, table_name, fields, create_id=False, schema=None):
    # Create a MetaData object without the 'bind' argument
    metadata = sqlalchemy.MetaData()

    # Create the Table object without the 'bind' argument
    table = sqlalchemy.Table(table_name, metadata, schema=schema)

    # Bind the table to the engine using the 'bind' method
    table.bind = connectable

    # Rest of your code...

# The rest of your code remains unchanged
create_table_from_csv(engine,
                      "IBRD_Balance_Sheet__FY2010.csv",
                      table_name="ibrd_balance",
                      fields=[
                          ("category", "string"),
                          ("category_label", "string"),
                          ("subcategory", "string"),
                          ("subcategory_label", "string"),
                          ("line_item", "string"),
                          ("year", "integer"),
                          ("amount", "integer")],
                      create_id=True
                  )


In [11]:
from cubes import Workspace
workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")

In [13]:
workspace.import_model("model.json")

In [14]:
browser = workspace.browser("irbd_balance")

TypeError: __init__() got an unexpected keyword argument 'bind'

# Using PyCubes

In [ ]:
%pip install pycubes

In [ ]:
%pip show pucubes

In [ ]:
from pycubes import Dimension, Hierarchy

# Define dimensions
time = Dimension("Time", levels=["Year", "Quarter", "Month"])
product = Dimension("Product", levels=["Category", "Subcategory"])


In [ ]:
from pycubes import PointCut, Cell, Cube

# Define the cube
cube = Cube("SalesCube", dimensions=[time, product])

# Add a measure
cube.add_edge(["Revenue", "Amount"])


In [ ]:
# Load data into the cube
data = {
    (2019, "Q1", "Laptops", "Gaming"): {"Revenue": 500000},
    (2019, "Q2", "Laptops", "Gaming"): {"Revenue": 550000},
    # Add more data points...
}

for point, values in data.items():
    cell = Cell(cube, PointCut(cube, **dict(zip(cube.dimension_names, point))))
    cell.data.update(values)

In [ ]:
# Aggregate data
cube.aggregate()


In [ ]:
# Query the cube
result = cube.query(
    drilldown=["Time.Year", "Product.Category"],
    cuts=[("Time.Year", "2019")]
)

for cell in result.cells:
    print(cell, cell.data)
